# Wine Price Prediction - Model Deployment


## Data and Use case

[**Wine Reviews** - 130k wine reviews with variety, location, winery, price, and description](https://www.kaggle.com/zynicide/wine-reviews/home)

This dataset is available on Kaggle and contains around 130k of wine reviews. The data was scraped from [WineEnthusiast](http://www.winemag.com/?s=&drink_type=wine) on November 22nd, 2017.

I plan to use this dataset to develop a model that predicts wine price for specified set of parameters, like wine variety, region, desired quality. Such model, may be integrated into an application that runs on a mobile device to suggest price range during wine shopping without need to do online search.

## Deploy our Keras model in WML

First, install the Python API client from pypi.

In [ ]:
# !rm -rf $PIP_BUILD/watson-machine-learning-client

In [ ]:
#Install the WML client if required.
# !pip install watson-machine-learning-client --upgrade
# !pip install wget

Authenticate to the WML service on IBM Cloud.

In [ ]:
# The code was removed by Watson Studio for sharing.

In [ ]:
wml_credentials={
  "url": service_url,
  "username": user,
  "password": password,
  "instance_id": instance_id
}

Instantiate WML client.

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
wml_client = WatsonMachineLearningAPIClient(wml_credentials)

Details about the WML service instance are available in service_instance.details.

In [ ]:
# List information about your WML service instance.
print(wml_client.service_instance.details)

In [ ]:
# List information about the stored models.
wml_client.repository.list_models()

In [ ]:
# List the deployed models.
wml_client.deployments.list()

Let's prepare archive with our model for upload.

In [ ]:
!tar -cvzf wine-model.tgz wine-price-prediction.model.keras.final.fitted.h5

Store the archive file in the WML service repository, then list all the files stored in the repository.

In [ ]:
# Save the downloaded file to the WML repository.
props = {wml_client.repository.ModelMetaNames.NAME: 'Wine Price Prediction',
              wml_client.repository.ModelMetaNames.AUTHOR_NAME: "Oleg Kholod", 
              wml_client.repository.ModelMetaNames.FRAMEWORK_NAME: "tensorflow", 
              wml_client.repository.ModelMetaNames.FRAMEWORK_VERSION: "1.5"}

model_details = wml_client.repository.store_model('wine-model.tgz', props)
wml_client.repository.list_models()

In [ ]:
# Create the online deployment.
model_uid = wml_client.repository.get_model_uid(model_details)
model_deployment = wml_client.deployments.create(model_uid, name = 'Wine Price Prediction')

wml_client.deployments.list()

In [ ]:
# Verify the online scoring endpoint.
model_scoring_url = wml_client.deployments.get_scoring_url(model_deployment)

print(model_scoring_url)

## Check the deployed model

Let's check the prediction by the model deployed to WML and the same model running in Watson Studio. I use a dummy data in both cases, we just need to see the same result.

In [ ]:
import numpy as np

x = np.zeros((1146,), dtype=float)

# Score data and predict price.
scoring_data = {'fields': ['inputs'], 'values': [np.array(x).tolist()]}
predictions = wml_client.deployments.score(model_scoring_url, scoring_data) 

print(predictions)

Run the model in Watson Studio.

In [ ]:
from keras.models import load_model
model = load_model('wine-price-prediction.model.keras.final.fitted.h5')

In [ ]:
model.predict(np.zeros((1,1146), dtype=float))

We can see that both calls produced the same result.